In [1]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [ ]:
conn = sqlite3.connect('data.sqlite3')

In [ ]:
faculties = pd.read_sql_query('''
select * from faculties
''', con=conn)

In [ ]:
usubs = pd.read_sql_query('''
select * from users_subscriptions 
where "type"='page'
''', con=conn)

In [ ]:
ufacs = pd.read_sql_query('''
select * from users_faculties
''', con=conn)

In [ ]:
# remove duplicates
user_fac = ufacs[(~ufacs[['user_id', 'faculty_id']].duplicated())
      & (ufacs['faculty_id'].isin(faculties['id'].astype(str)))
     ][['user_id', 'faculty_id']]

In [ ]:
usubs['user_id'] = usubs['user_id'].astype(int)
user_fac['user_id'] = user_fac['user_id'].astype(int)

In [ ]:
usr_sub_fac = usubs[['user_id', 'subs_id']].merge(user_fac)

In [ ]:
sub_fac = usr_sub_fac[~usr_sub_fac[['faculty_id', 'subs_id']].duplicated()][['faculty_id', 'subs_id']]

In [ ]:
sub_fac

In [ ]:
# sub_fac.to_csv('sub_fac.csv', index=False, encoding='utf-8')

In [96]:
ohe = OneHotEncoder(sparse=False, dtype=int)

In [99]:
labeld = []
result = []

In [100]:
counter = 0
for chunk in pd.read_csv('sub_fac.csv', chunksize=5000):
    print(f'chunk started')
    print(chunk.index)
    chunk = chunk.reset_index(drop=True)
    print(chunk.index)
    fitted = ohe.fit_transform(chunk[['subs_id']])
    labeled = pd.concat([chunk[['faculty_id']].reindex(),
                         pd.DataFrame(fitted, columns=ohe.categories_[0])], axis=1)
    labeld.append(labeled)
    ptable = labeled.pivot_table(index='faculty_id',
                                 values=[c for c in labeled.columns if c != 'faculty_id'], aggfunc='sum')
    #result = pd.concat([result, ptable], sort=True)
    result.append(ptable.reset_index())
    counter += 1
    print('chunk processed. elements: {}\n'.format(len(result)))
#     if counter % 5 == 0:
#         print('\npivoting\n')
#         result = result.pivot_table(index='faculty_id',
#                                    values=[c for c in result.columns if c != 'faculty_id'], aggfunc='sum')

chunk started
RangeIndex(start=0, stop=5000, step=1)
RangeIndex(start=0, stop=5000, step=1)
chunk processed. elements: 1

chunk started
RangeIndex(start=5000, stop=10000, step=1)
RangeIndex(start=0, stop=5000, step=1)
chunk processed. elements: 2

chunk started
RangeIndex(start=10000, stop=15000, step=1)
RangeIndex(start=0, stop=5000, step=1)
chunk processed. elements: 3

chunk started
RangeIndex(start=15000, stop=20000, step=1)
RangeIndex(start=0, stop=5000, step=1)
chunk processed. elements: 4

chunk started
RangeIndex(start=20000, stop=25000, step=1)
RangeIndex(start=0, stop=5000, step=1)
chunk processed. elements: 5

chunk started
RangeIndex(start=25000, stop=30000, step=1)
RangeIndex(start=0, stop=5000, step=1)
chunk processed. elements: 6

chunk started
RangeIndex(start=30000, stop=35000, step=1)
RangeIndex(start=0, stop=5000, step=1)
chunk processed. elements: 7

chunk started
RangeIndex(start=35000, stop=40000, step=1)
RangeIndex(start=0, stop=5000, step=1)
chunk processed. ele

In [104]:
allin = pd.concat(result, sort=True).fillna(0)

In [107]:
allin = allin.pivot_table(index='faculty_id',
                         values=[c for c in allin.columns if c != 'faculty_id'], aggfunc='sum')

In [112]:
allin = allin.reset_index()

In [160]:
allin.to_csv('pivot_ohenot.csv', index=False)

In [150]:
allin = allin.astype(int)

In [152]:
clf = GaussianNB()

In [153]:
allin_x = allin.drop(columns='faculty_id')
allin_y = allin['faculty_id']

In [154]:

X_train, X_test, y_train, y_test = train_test_split(allin_x, allin_y)

In [171]:
clf.fit(X_train, y_train)

In [172]:
clf.score(X_test, y_test)

0.0

In [167]:
clf.predict([X_test.loc[X_test.index[0]]])

array([2231285])

In [168]:
y_test

32    2266965
0        2470
10     203338
4        2961
20    2173844
25    2231268
9      106511
31    2231790
28    2231283
Name: faculty_id, dtype: int32

In [169]:
allin

,faculty_id,27,28,30,41,47,65,83,158,159,...,202723727,202729930,202737287,202741307,202785068,202805101,202805887,202805889,202820900,202839873
0,2470,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2951,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,2957,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2959,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2961,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,2973,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,20422,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,40378,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,40773,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,106511,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [170]:
from sklearn.linear_model import LogisticRegression

In [193]:
logreg = LogisticRegression()

In [174]:
from sklearn.decomposition import PCA

In [187]:
pca = PCA()

In [188]:
pca.fit(X_train)

PCA()

In [190]:
pca.n_components_

25

In [191]:
PCAX_train = pca.transform(X_train)

In [192]:
PCAX_test = pca.transform(X_test)

In [194]:
logreg.fit(PCAX_train, y_train)

LogisticRegression()

In [195]:
logreg.score(PCAX_test, y_test)

0.0

In [196]:
logreg.predict(PCAX_test)

array([2231285, 2161061, 2231285, 2231276, 2139044, 2231285,    2951,
       2218244, 2231285])

In [197]:
y_test

32    2266965
0        2470
10     203338
4        2961
20    2173844
25    2231268
9      106511
31    2231790
28    2231283
Name: faculty_id, dtype: int32

In [3]:
fac_sub = pd.read_csv('fac_sub.csv')

In [4]:
subs = fac_sub.drop(columns='faculty_id')

In [11]:
ohe = OneHotEncoder(categories=subs.columns,
                   sparse=False,
                    dtype=int,
                   handle_unknown='ignore')

In [12]:
ohe

OneHotEncoder(categories=Index(['27', '28', '41', '47', '65', '272', '299', '337', '439', '442',
       ...
       '202588677', '202591514', '202597077', '202632157', '202663824',
       '202682637', '202701885', '202729930', '202737287', '202839873'],
      dtype='object', length=44998),
              dtype=<class 'int'>, handle_unknown='ignore', sparse=False)

In [13]:
import pickle

In [14]:
with open('ohe_subs.pickle', 'wb') as f:
    pickle.dump(ohe, f)

In [19]:
list(ohe.categories)

['27',
 '28',
 '41',
 '47',
 '65',
 '272',
 '299',
 '337',
 '439',
 '442',
 '516',
 '613',
 '765',
 '828',
 '851',
 '932',
 '983',
 '1118',
 '1552',
 '1670',
 '1959',
 '1980',
 '2002',
 '2251',
 '2805',
 '2856',
 '2904',
 '3168',
 '3305',
 '4097',
 '4326',
 '4391',
 '4569',
 '4829',
 '5333',
 '5562',
 '5726',
 '6039',
 '6386',
 '6585',
 '6783',
 '7057',
 '7275',
 '8564',
 '9117',
 '9225',
 '9525',
 '9720',
 '9768',
 '9928',
 '10405',
 '11614',
 '11787',
 '11814',
 '11817',
 '11999',
 '12390',
 '14114',
 '14825',
 '15898',
 '16685',
 '17036',
 '17228',
 '17708',
 '17835',
 '18257',
 '19038',
 '21142',
 '25176',
 '25457',
 '25624',
 '27771',
 '29346',
 '30281',
 '31687',
 '35005',
 '35408',
 '37323',
 '37756',
 '41524',
 '41812',
 '42959',
 '45130',
 '45734',
 '46526',
 '47987',
 '48107',
 '48350',
 '48564',
 '48864',
 '48978',
 '49648',
 '49784',
 '52105',
 '53269',
 '53474',
 '54079',
 '56677',
 '57716',
 '61972',
 '63273',
 '64348',
 '65860',
 '66298',
 '68526',
 '69915',
 '69961',
 '